In [1]:

import numpy as np
import pandas as pd
import math
from itertools import combinations
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
import getpass
username= getpass.getuser()
counter = 0

directory = 'C:/Users/'+ username +'/Downloads'
shapedir= 'D:\Shapefiles_for_Studyarea\Shapefiles'


ModuleNotFoundError: No module named 'rasterio'

In [ ]:
# !pip install geemap

In [ ]:
#!pip install earthengine-api

import ee
import geemap
ee.Initialize()

In [ ]:
from ipywidgets import AppLayout
import ipywidgets as widgets
import rasterio
from rasterio.plot import show

In [ ]:
#Date picker 
dateBox = widgets.DatePicker(
    description='',
    disabled=False
)

Map = geemap.Map()


legend_keys = ['<=0', '0-0.2', '0.2-0.4', '0.4-0.6', '>0.6']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = ['654321','FFA500','FFFF00', '00FF00', '008000']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='right'
)

#Map
mapWidget = widgets.Output()
with mapWidget:
    display(Map)
  
    
#labels
heading=widgets.Label('RaoQ Index Calculator')
step1 = widgets.HTML(value="<h4>Step 1: Select the date</h4>")
step2 = widgets.HTML(value='<h4>Step 2: Select Region of Interest</h4>')
step2or=widgets.HTML(value='<h4>Select Predefined Hyrcanian Forests</h4>')
step3 = widgets.HTML(value='<h4>Step 3: Select Dataset</h4>')
step4=   widgets.HTML(value='<h4>Step 4: Select Index</h4>')
step5= widgets.HTML('<h4>Step 5: Compute RaoQ output</h4>')
Display=widgets.HTML('<h4>Step 6:Display NDVI or RaoQ Output')
step7=widgets.HTML('<h4>Step 7: Change in Index over time</h4>')
step8=widgets.HTML('<h4>                                               </h4>')

startd=widgets.DatePicker(
     description='FROM :',
     disabled=False
)
endd=widgets.DatePicker(
     description='TO :',
     disabled=False
)

changeout=widgets.Output()


douts=widgets.Output()
Select_Output = widgets.Dropdown(
    options=['RaoQ','NDVI'],
    description='',
    disabled=False,
)



#Buttons
getROI = widgets.Button(description='Ok')
DownloadRao_Btn = widgets.Button(description='Compute RaoQ')
Select_Dataset = widgets.Dropdown(
    options=['LANDSAT','MODIS','SENTINEL-2'],
    value='MODIS',
    description='',
    disabled=False,
)

Select_Forest = widgets.Dropdown(
    options=['None','Golestan-1', 'Golestan-2','Abr','Jahan Nama', 'Boola','Vaz-1', 'Vaz-2','Kojoor','Chahar Bagh','Koshk-e-Dharan','Siahud-e-Raudbar','Gasht-Raudkhan','Lisar'],
    value='None',
    description='',
    disabled=False,
)
# Dict = {'Golestan-1':X1 , 'Golestan-2' : X2 ,'Abr' : X3 ,'Jahan Nama':X5 , 'Boola' :X6 ,'Vaz-1':X8, 'Vaz-2':X9 ,'Kojoor' :X10,'Chahar Bagh':X11,'Koshk-e-Dharan':X12,'Siahud-e-Raudbar':X13,'Gasht-Raudkhan':X14,'Lisar':X15}


# #Text Area
# estimate_yield_Out = widgets.Textarea(
#     value='',
#     placeholder='',
#     description='',
#     disabled=True,
#     layout={'height': '300px'}
# )

#Display RaoQ image 
estimate_yield_plot = widgets.Output()


In [ ]:
import numpy as np
def spectralrao(data_input, output_path, distance_m="euclidean", p=np.nan, window=9, mode="classic", na_tolerance=0.0,
                simplify=1, debugging=False):
    def euclidean_distance(pair_list):
        #         Compute euclidean distance between two vectors
        tmp = [(x[0] - x[1]) ** 2 for x in pair_list]

        return math.sqrt(sum(tmp))

    def mana_distance(pair_list):
        # compute manatthan distance between two vectors
        tmp = [abs(x[0] - x[1]) for x in pair_list]
        return sum(tmp)

    def compute_isNaN(inputArr, valueOfNan=1):
        # Return an array with valueOFNan instead of the NaN of the input array
        inputArr = inputArr.ravel()
        outArr = np.zeros(len(inputArr))
        for i in range(len(inputArr)):
            if inputArr[i] != valueOfNan:
                outArr[i] = 1
        return outArr

    def tiff_to_numpy(tiff_input):
        # Convert Tiff input tu Numpy
        matrix1 = tiff_input.read()
        matrix1 = matrix1.reshape((matrix1.shape[1]), matrix1.shape[2])
        # print(matrix1)
        minNum = -999
        matrix1[matrix1 == minNum] = np.nan
        return matrix1

    def export_tiff(naip_meta, output_rao, output_path):
        # Write the computation output on Tiff file
        naip_transform = naip_meta["transform"]
        naip_crs = naip_meta["crs"]
        naip_transform, naip_crs
        naip_meta['count'] = 1
        naip_meta['dtype'] = "float64"
        with rasterio.open(output_path, 'w', **naip_meta) as dst:
            dst.write(output_rao, 1)

    integer_dtypes = ("int8", "int16", "int32", "int64", "uint8", "uint16", "uint32", "uint64")
    float_dtypes = ("float_", "float16", "float32", "float64")

    if mode == "classic":  # one dimension input
        info = data_input.profile
        data_input = tiff_to_numpy(data_input)
        rasterm = np.copy(data_input)

        isfloat = False  # If data are float numbers, transform them in integer
        if (rasterm.dtype.name not in integer_dtypes):
            print("Converting input data in an integer matrix...")
            isfloat = True
            mfactor = np.power(1000000, simplify)
            rasterm = rasterm * mfactor
            rasterm = rasterm.astype('int64')
        #           print(rasterm)

        else:
            rasterm = rasterm.astype('int64')
        #   rasterm = np.transpose(rasterm)
    if window % 2 == 1:
        w = int((window - 1) / 2)
    else:
        raise Exception("The size of moving window must be an odd number. Exiting...")

    # Rao_Q initialization to NaN
    if type(data_input) is list:
        print(True)
        raoqe = np.zeros(shape=data_input[0].shape)
    else:
        raoqe = np.zeros(shape=rasterm.shape)
    raoqe[:] = np.nan

    # Check if there are NaN value which becomes int
    nan_to_int64 = np.array([np.nan]).astype('int64')[0]

    if mode == 'classic':  # one dimension input
        intNaNPresence = nan_to_int64 in rasterm
        # Reshape values
        s = pd.Series(rasterm.flatten('F'), dtype="category")
        values = [s.cat.codes[i] + 1 for i in range(len(s.cat.codes))]
        # print(values)
        rasterm_1 = np.array(
            [[values[x + y * rasterm.shape[0]] for y in range(rasterm.shape[1])] for x in range(rasterm.shape[0])])

        # Add fake columns and rows for moving window
        trasterm = np.zeros(shape=(rasterm.shape[0] + 2 * w, rasterm.shape[1] + 2 * w))
        trasterm[:] = np.nan
        trasterm[w:w + rasterm.shape[0], w:w + rasterm.shape[1]] = rasterm_1[:]
        # print(trasterm)

        # Derive distance matrix
        classes_values = np.array(s.cat.categories)
        # print("------------------------------------")
        # print(classes_values)
        nan_to_int64 = np.array([np.nan]).astype('int64')[0]

        # Loop over each pixel
        for cl in range(w, w + rasterm.shape[1]):
            for rw in range(w, w + rasterm.shape[0]):
                #print('Computing pixel {:d},{:d}'.format(rw, cl))

                # Check if the pixel is on the border
                borderCondition = np.sum(
                    np.invert(np.isnan(trasterm[rw - w:rw + w + 1, cl - w:cl + w + 1]))) < np.power(window, 2) - (
                                          (np.power(window, 2)) * na_tolerance)
                # Check if there are too many nans in the window
                tooManyIntNan = np.sum(compute_isNaN(trasterm[rw - w:rw + w + 1, cl - w:cl + w + 1])) < np.power(window,
                                                                                                                 2) - (
                                        (np.power(window, 2)) * na_tolerance)
                if (intNaNPresence and tooManyIntNan) or borderCondition:
                    # It is a pixel to jump if it is on the border or if there are nans and too many in the window
                    # print('Skipping pixel {:d},{:d}'.format(rw,cl))
                    pass
                else:  # Pixel to execute                    
                    tw = pd.Series(trasterm[rw - w:rw + w + 1, cl - w:cl + w + 1].flatten('F'),
                                   dtype="category").value_counts(ascending=True)
                    tw_values = np.array(tw.values)
                    tw_labels = np.array(tw.index)

                    # to use if value_counts parameter dropna is set to False (default True)
                    if np.sum(np.isnan(np.array(tw.index))) > 0:
                        pass  # here you have to drop the na entry, but pd.Series handles it on its own

                    if debugging:
                        print("Working on coords ", rw, ",", cl, ". classes length: ", len(tw_values), ". window size=",
                              window)

                    # if clause to exclude windows with only 1 category
                    if (len(tw_values) < 2):
                        pass  # THere is only the same value in the window then skip
                    else:
                        if (nan_to_int64 in tw_labels):
                            temp_index = np.where(tw_labels == nan_to_int64)
                            p = tw_values / (np.sum(tw_values) - tw_values[index])
                            p[temp_index] = 0
                        else:
                            p = tw_values / np.sum(tw_values)
                        p1 = np.zeros(shape=(len(tw_labels), len(tw_labels)))
                        for r in range(len(tw_labels)):
                            for c in range(r + 1, len(tw_labels)):
                                p1[r, c] = p[c] * p[r]

                        # Find the indices of the classes to use from distance matrix d1 to build d2
                        indices = np.array([int(el) - 1 for el in tw_labels])

                        # Compute value in the windows for only the current pixwl
                        d2 = np.zeros(shape=(len(indices), len(indices)))
                        for i, r in enumerate(indices):
                            for j, c in enumerate(indices):
                                if ((classes_values[r] == nan_to_int64) | (
                                        classes_values[c] == nan_to_int64)):  # distanza 0 se un valore e' nan
                                    d2[i, j] = 0
                                else:
                                    d2[i, j] = classes_values[c] - classes_values[r]

                        # Finally compute RAO for the current pixel
                        if isfloat:
                            raoqe[rw - w, cl - w] = np.sum(p1 * d2) / mfactor
                        else:
                            raoqe[rw - w, cl - w] = np.sum(p1 * d2)

        # End of for loop 
        print(("\nCalculation of Rao's index complete.\n"))
        # End of classic Rao computation


    elif mode == 'multidimension':  # multi dimensions input
        info = data_input[0].profile
        # Convertion to Numpy and add to the same list
        numpy_data = []
        for inpu in data_input:
            numpy_data.append(tiff_to_numpy(inpu))
        if debugging:
            print("check: Into multidimensional clause")
        trastersm_list = []
        for mat in numpy_data:
            trasterm = np.zeros(shape=(mat.shape[0] + 2 * w, mat.shape[1] + 2 * w))
            trasterm[:] = np.nan
            trasterm[w:w + mat.shape[0], w:w + mat.shape[1]] = mat
            trastersm_list.append(trasterm)

        # Iterate over each pixel
        for cl in range(w, numpy_data[0].shape[1] + w):
            for rw in range(w, numpy_data[0].shape[0] + w):
                print('Computing pixel {:d},{:d}'.format(rw, cl))
                borderCondition = [
                    np.sum(np.invert(np.isnan(x[rw - w:rw + w + 1, cl - w:cl + w + 1]))) < np.power(window, 2) - (
                            (np.power(window, 2)) * na_tolerance) for x in trastersm_list]
                if True in borderCondition:
                    # Skipping the pixel which does not respect border condition (both NaN tollerance and distance to border)
                    pass
                else:  # Execute the RAO for the current pixel
                    tw = [x[rw - w:rw + w + 1, cl - w:cl + w + 1] for x in
                          trastersm_list]  # compute the window for each pixel
                    lv = [x.ravel() for x in tw]  # linearize the matrices

                    # Compute all the possibile point combination
                    vcomb = combinations(range(lv[0].shape[0]), 2)
                    vcomb = list(vcomb)
                    vout = []
                    # Compute value for each combination
                    for comb in vcomb:
                        lpair = [[x[comb[0]], x[comb[1]]] for x in lv]
                        if distance_m == 'euclidean':  # Distance option is Euclidean
                            out = euclidean_distance(lpair)
                        elif distance_m == 'mana':  # Distance option is manattan
                            out = man_distance(lpair)
                        vout.append(out)
                    # Rescale output
                    vout_rescaled = [x * 2 for x in vout]
                    vout_rescaled[:] = [x / window ** 4 for x in vout_rescaled]
                    # Final RAO computation for current pixel
                    raoqe[rw - w, cl - w] = np.nansum(vout_rescaled)

    output = [raoqe]
    export_tiff(info, raoqe, output_path)
    return output


In [ ]:
    import os
   # !pip install colorama
    import colorama
    from colorama import Fore
    from IPython.display import clear_output

In [ ]:
index = widgets.Dropdown(
                      options=['NDVI','EVI'],
                      value='NDVI',
                      description='',
                      disabled=False,)

var=Select_Dataset.value
lookhere=index.value
outs = widgets.Output()
def mytest2(x):
    global var
    global lookhere
    var=Select_Dataset.value
    lookhere=index.value
    
    with outs:
        clear_output()
        
        if (index.value=='EVI' and Select_Dataset.value=='MODIS'):
            print(Fore.RED + "Sorry! EVI index is not a valid selection"+'\n'+"for MODIS dataset")


Select_Dataset.observe(mytest2, names='value')

def mytest(x):
    global var
    
    global lookhere
    lookhere=index.value
    var=Select_Dataset.value
    
    with outs:
        clear_output()
        
        if (index.value=='EVI' and Select_Dataset.value=='MODIS'):
            print(Fore.RED + "Sorry! EVI index is not a valid selection"+'\n'+"for MODIS dataset")
    


index.observe(mytest, names='value')










In [ ]:
studyarea='D:\Shapefiles_for_Studyarea\Hyrcanian_Forest.shp'
studyarea1='D:\Shapefiles_for_Studyarea\Hyrcanian_Forest_individual.shp'
studya=geemap.shp_to_ee(studyarea)

# Map.addLayer(studya,{},'StudyArea')

studya1=geemap.shp_to_ee(studyarea1)

# Map.addLayer(studya,{},'StudyArea')

In [ ]:
X1=shapedir + "\X1_Golestan.shp"
X1=geemap.shp_to_ee(X1)
Map.addLayer(X1,{},'X1')

X2=shapedir + "\X2_Golestan.shp"
X2=geemap.shp_to_ee(X2)
Map.addLayer(X2,{},'X2')

X3=shapedir + "\X3_Abr.shp"
X3=geemap.shp_to_ee(X3)
Map.addLayer(X3,{},'X3')

X5=shapedir + "\X5_JahanNama.shp"
X5=geemap.shp_to_ee(X5)
Map.addLayer(X5,{},'X5')

X6=shapedir + "\X6_Boola.shp"
X6=geemap.shp_to_ee(X6)
Map.addLayer(X6,{},'X6')

X8=shapedir + "\X8_Vaz.shp"
X8=geemap.shp_to_ee(X8)
Map.addLayer(X8,{},'X8')

X9=shapedir + "\X9_Vaz.shp"
X9=geemap.shp_to_ee(X9)
Map.addLayer(X9,{},'X9')

X10=shapedir + "\X10_Kojoor.shp"
X10=geemap.shp_to_ee(X10)
Map.addLayer(X10,{},'X10')

X11=shapedir + "\X11_Chahar_Bagh.shp"
X11=geemap.shp_to_ee(X11)
Map.addLayer(X11,{},'X11')

X12=shapedir + "\X12_Koshk_e_Dharan.shp"
X12=geemap.shp_to_ee(X12)
Map.addLayer(X12,{},'X12')

X13=shapedir + "\X13_Siahud-e-Raudbar.shp"
X13=geemap.shp_to_ee(X13)
Map.addLayer(X13,{},'X13')

X14=shapedir + "\X14_Gasht-Raudkhan.shp"
X14=geemap.shp_to_ee(X14)
Map.addLayer(X14,{},'X14')

X15=shapedir + "\X15_Lisar.shp"
X15=geemap.shp_to_ee(X15)
Map.addLayer(X15,{},'X15')

Dict = {'Golestan-1':X1 , 'Golestan-2' : X2 ,'Abr' : X3 ,'Jahan Nama':X5 , 'Boola' :X6 ,'Vaz-1':X8, 'Vaz-2':X9 ,'Kojoor' :X10,'Chahar Bagh':X11,'Koshk-e-Dharan':X12,'Siahud-e-Raudbar':X13,'Gasht-Raudkhan':X14,'Lisar':X15}










In [ ]:


legend_keys = ['<=0', '0-0.2', '0.2-0.4', '0.4-0.6', '>0.6']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = ['654321','FFA500','FFFF00', '00FF00', '008000']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='topright'
)



In [ ]:


def DownloadRaoq(change):
    
    global counter
    counter = counter + 1  
    feature = Map.draw_last_feature
    ROI =0
    filtersize=window.value
    if(Select_Forest.value!='None'):
        ROI=Dict[Select_Forest.value]
        Map.centerObject(ROI, 11)
        
    else:
        ROI = feature.geometry()

    #selected dataset
    dataset_selected=Select_Dataset.value
    index_selected=index.value

    
    def maskS2clouds(image):
        qa = image.select('QA60')
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
        return image.updateMask(mask).divide(10000) 
    
    
    year  = dateBox.value.year
    month = dateBox.value.month
    day   = dateBox.value.day
    period_of_interest = ee.Filter.date(str(year) + "-"+ str(month) + "-" + str(1),str(year) +"-"+str(month)+"-"+ str(30))
    
    
    
    #Sentinel

    inBands = ["B2","B3","B4","B5","B6","B7","B8","B8A","B11"]
    outBands = inBands + ["NDVI","EVI"]
    
    #Landsat
    
    inBandsl = ["SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6","SR_B7","SR_QA_AEROSOL"]
    outBandsl = inBands + ["NDVI","EVI"]
    
    
    #Sentinel
    def addNDVI(image):
        ndvi = image.expression('((NIR - RED) / (NIR + RED))',  {'NIR': image.select("B8"), 'RED': image.select("B4")}) \
                   .rename('NDVI') \
                .copyProperties(image,['system:time_start'])

        return image.addBands(ndvi)

   
    def addEVI(image):
        evi = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',  {'NIR': image.select("B8"), 'RED': image.select("B4"), 'BLUE': image.select("B2")}) \
                   .rename('EVI') \
                   .copyProperties(image,['system:time_start'])
        return image.addBands(evi)
            
            
    #LANDSAT
    def addNDVIl(image):
        ndvi = image.expression('((NIR - RED) / (NIR + RED))',  {'NIR': image.select("SR_B5"), 'RED': image.select("SR_B4")}) \
                   .rename('NDVI') \
        .copyProperties(image,['system:time_start'])
        return image.addBands(ndvi)
    
    def addEVIl(image):
        evi = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',  {'NIR': image.select("SR_B5"), 'RED': image.select("SR_B4"), 'BLUE': image.select("SR_B2")}) \
              .rename('EVI') \
              .copyProperties(image,['system:time_start'])
        
        return image.addBands(evi)
        
        
    
    #Modis
    if(dataset_selected=='MODIS'):
     
            dataset_modis = ee.ImageCollection('MODIS/MOD09GA_006_NDVI') \
                           .filter(period_of_interest) \
                           .filterBounds(ROI) \

            colorized = dataset_modis.select('NDVI') 
            

            ndvi_modis=colorized.first()
            ndvi_modis=ndvi_modis.clipToBoundsAndScale(ROI)
            ndvic=ndvi_modis
            thresholds = ee.Image([0.0, 0.2, 0.4, 0.6,0.8]); 
            ndvic = ndvic.gt(thresholds).reduce('sum').toInt()
            ndvic = ndvic.clipToBoundsAndScale(ROI)
            ndviParams =  {'min': 1, 'max':5, 'palette' : ['654321','FFA500','FFFF00', '00FF00', '008000']}
            Map.addLayer(ndvic,ndviParams, 'Classified NDVI',opacity=0.85)
            

#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            out_file = directory + '/ndvi'+str(counter)+'.tif'
            geemap.ee_export_image(ndvi_modis, filename=out_file)
            ndvi=rasterio.open(directory + '/ndvi'+str(counter)+'.tif') 
            output=spectralrao(ndvi,directory + '/RaoQ_C'+dataset_selected+'_'+index_selected+'.tif', window=filtersize, na_tolerance=0)
    
    
    if(dataset_selected=='LANDSAT'):
     
        dataset_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
                     .filter(period_of_interest) \
                     .filterBounds(ROI) \
                     .first()
        
        ndvil = dataset_landsat.normalizedDifference(['B5', 'B4']).rename('NDVI')
              
        evil=dataset_landsat.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',  {'NIR': dataset_landsat.select("B5"), 'RED': dataset_landsat.select("B4"), 'BLUE': dataset_landsat.select("B2")})
        
         #Landsat
        ndvil = ndvil.clipToBoundsAndScale(ROI,1000,1000)   
        ndvic=ndvil
        thresholds = ee.Image([0.0, 0.2, 0.4, 0.6,0.8]); 
        ndvic = ndvic.gt(thresholds).reduce('sum').toInt()
        ndvic = ndvic.clipToBoundsAndScale(ROI,1000,1000)
        ndviParams =  {'min': 1, 'max':5, 'palette' : ['654321','FFA500','FFFF00', '00FF00', '008000']}
        Map.addLayer(ndvic,ndviParams, 'Classified NDVI',opacity=0.85)
        
       
        
        if(index_selected=='NDVI') :
             
                            
                          
                          

#                             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                            out_file = directory + '/ndvi'+str(counter)+'.tif'
                            geemap.ee_export_image(ndvil, filename=out_file)
                            ndvi=rasterio.open(directory + '/ndvi'+str(counter)+'.tif') 
                            output=spectralrao(ndvi,directory + '/RaoQ_C'+dataset_selected+'_'+index_selected+'.tif', window=filtersize, na_tolerance=0)

        else:
            
            
            evil = evil.clipToBoundsAndScale(ROI,1000,1000)
        
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            out_file = directory + '/evi'+str(counter)+'.tif'
            geemap.ee_export_image(evil, filename=out_file)
            evi=rasterio.open(directory + '/evi'+str(counter)+'.tif') 
            output=spectralrao(evi,directory + '/RaoQ_C'+dataset_selected+'_'+index_selected+'.tif', window=filtersize, na_tolerance=0)


        
        
   #Sentinel
    if(dataset_selected=='SENTINEL-2'):
       
        dataset_sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
                     .filter(period_of_interest) \
                     .filterBounds(ROI) \
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                     .map(maskS2clouds)
    
    
        collection = dataset_sentinel.select(inBands).map(addNDVI).map(addEVI)

         #Sentinel
        band_median = collection.select(inBands).median()

         #Sentinel
        ndvimean = collection.select('NDVI').reduce(ee.Reducer.mean()).rename("NDVI")
        evimean = collection.select('EVI').reduce(ee.Reducer.mean()).rename("EVI")
        

        #sentinel
        ndvimean = ndvimean.clipToBoundsAndScale(ROI,1000,1000)
        evimean =  evimean.clipToBoundsAndScale(ROI,1000,1000)
        
        ndvic=ndvimean
        thresholds = ee.Image([0.0, 0.2, 0.4, 0.6,0.8]); 
        ndvic = ndvic.gt(thresholds).reduce('sum').toInt()
        ndvic = ndvic.clipToBoundsAndScale(ROI,1000,1000)
        ndviParams =  {'min': 1, 'max':5, 'palette' : ['654321','FFA500','FFFF00', '00FF00', '008000']}
        Map.addLayer(ndvic,ndviParams, 'Classified NDVI',opacity=0.85)
        
        
        if(index_selected=='NDVI') :
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            out_file = directory + '/ndvi'+str(counter)+'.tif'
            geemap.ee_export_image(ndvimean, filename=out_file)
            ndvi=rasterio.open(directory + '/ndvi'+str(counter)+'.tif') 
            output=spectralrao(ndvi,directory + '/RaoQ_C'+dataset_selected+'_'+index_selected+'.tif', window=filtersize, na_tolerance=0)
        else:
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            out_file = directory + '/evi'+str(counter)+'.tif'
            geemap.ee_export_image(evimean, filename=out_file)
            ndvi=rasterio.open(directory + '/evi'+str(counter)+'.tif') 
            output=spectralrao(ndvi,directory + '/RaoQ_C'+dataset_selected+'_'+index_selected+'.tif', window=filtersize, na_tolerance=0)
    
    fp2 = (directory + '/RaoQ_C'+Select_Dataset.value+'_'+index.value+'.tif')
    img2 = rasterio.open(fp2)
    fp = (directory + '/ndvi'+str(counter)+'.tif')
    img = rasterio.open(fp)
    
    with douts:
        clear_output()
        
        if (Select_Output.value=='NDVI' and counter>=1):
            
            display(show(img))
            
        elif(Select_Output.value=='RaoQ'):
            
            display(show(img2))

   

In [ ]:
def mytest3(x): 
    fp2 = (directory + '/RaoQ_C'+Select_Dataset.value+'_'+index.value+'.tif')
    img2 = rasterio.open(fp2)
    fp = (directory + '/ndvi'+str(counter)+'.tif')
    img = rasterio.open(fp)
    
    with douts:
        clear_output()
        
        if (Select_Output.value=='NDVI' and counter>=1):
            
            display(show(img))
            
        elif(Select_Output.value=='RaoQ'):
            
            display(show(img2))


Select_Output.observe(mytest3, names='value')

def mytest4(x):         
        if (Select_Forest.value!='None'):
            Map.centerObject(Dict[Select_Forest.value],11)
            
            
       


    
Select_Forest.observe(mytest4, names='value')

In [ ]:
DownloadRao_Btn.on_click(DownloadRaoq)
findchange = widgets.Button(description='Find Change')

In [ ]:
def Changeindex(change):
    import rasterio
    from rasterio.plot import show
    def tiff_to_numpy(tiff_input):
        # Convert Tiff input tu Numpy
        matrix1 = tiff_input.read()
        matrix1 = matrix1.reshape((matrix1.shape[1]), matrix1.shape[2])
        print(matrix1)
        minNum = -999
        matrix1[matrix1 == minNum] = np.nan
        return matrix1
       


    
    global counter
    counter = counter + 1  

    feature = Map.draw_last_feature
    ROI =0
    filtersize=window.value
    if(Select_Forest.value!='None'):
        ROI=Dict[Select_Forest.value]
        Map.centerObject(ROI, 11)
        
    else:
        ROI = feature.geometry()

    #selected dataset
    dataset_selected=Select_Dataset.value
    index_selected=index.value

    
    def maskS2clouds(image):
        qa = image.select('QA60')
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
        return image.updateMask(mask).divide(10000) 
    
    
    year1  = startd.value.year
    month1 = startd.value.month
    day1  = startd.value.day
    period_of_interest1 = ee.Filter.date(str(year1) + "-"+ str(month1) + "-" + str(1),str(year1) +"-"+str(month1)+"-"+ str(30))
    
    year2  = endd.value.year
    month2 = endd.value.month
    day2  = endd.value.day
    period_of_interest2 = ee.Filter.date(str(year2) + "-"+ str(month2) + "-" + str(1),str(year2) +"-"+str(month2)+"-"+ str(30))
    
    
    
    
    #Sentinel

    inBands = ["B2","B3","B4","B5","B6","B7","B8","B8A","B11"]
    outBands = inBands + ["NDVI","EVI"]
    
    #Landsat
    
    inBandsl = ["SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6","SR_B7","SR_QA_AEROSOL"]
    outBandsl = inBands + ["NDVI","EVI"]
    
    
    #Sentinel
    def addNDVI(image):
        ndvi = image.expression('((NIR - RED) / (NIR + RED))',  {'NIR': image.select("B8"), 'RED': image.select("B4")}) \
                   .rename('NDVI') \
                .copyProperties(image,['system:time_start'])

        return image.addBands(ndvi)

   
    def addEVI(image):
        evi = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',  {'NIR': image.select("B8"), 'RED': image.select("B4"), 'BLUE': image.select("B2")}) \
                   .rename('EVI') \
                   .copyProperties(image,['system:time_start'])
        return image.addBands(evi)
            
            
    #LANDSAT
    def addNDVIl(image):
        ndvi = image.expression('((NIR - RED) / (NIR + RED))',  {'NIR': image.select("SR_B5"), 'RED': image.select("SR_B4")}) \
                   .rename('NDVI') \
        .copyProperties(image,['system:time_start'])
        return image.addBands(ndvi)
    
    def addEVIl(image):
        evi = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',  {'NIR': image.select("SR_B5"), 'RED': image.select("SR_B4"), 'BLUE': image.select("SR_B2")}) \
              .rename('EVI') \
              .copyProperties(image,['system:time_start'])
        
        return image.addBands(evi)
        
        
    
    #Modis
    if(dataset_selected=='MODIS'):
     
            dataset_modis = ee.ImageCollection('MODIS/MOD09GA_006_NDVI') \
                           .filter(period_of_interest1) \
                           .filterBounds(ROI) \

            colorized = dataset_modis.select('NDVI') 
            

            ndvi_modis=colorized.first()
            ndvi_modis=ndvi_modis.clipToBoundsAndScale(ROI)
            
            
            dataset_modis2 = ee.ImageCollection('MODIS/MOD09GA_006_NDVI') \
                           .filter(period_of_interest2) \
                           .filterBounds(ROI) \

            colorized2 = dataset_modis2.select('NDVI') 
            

            ndvi_modis2=colorized2.first()
            ndvi_modis2=ndvi_modis2.clipToBoundsAndScale(ROI)
   
            
            change=ndvi_modis.subtract(ndvi_modis2)
                
            out_file2= directory + '/change'+str(counter)+'.tif'
            
            geemap.ee_export_image(change, filename=out_file2)
            with changeout:
                clear_output()
                
                fp3 = (directory + '/change'+str(counter)+'.tif')
                img3 = rasterio.open(fp3)
                display(show(img3))
#             ndvic=ndvi_modis
#             thresholds = ee.Image([0.0, 0.2, 0.4, 0.6,0.8]); 
#             ndvic = ndvic.gt(thresholds).reduce('sum').toInt()
#             ndvic = ndvic.clipToBoundsAndScale(ROI)
#             ndviParams =  {'min': 1, 'max':5, 'palette' : ['654321','FFA500','FFFF00', '00FF00', '008000']}
#             Map.addLayer(ndvic,ndviParams, 'Classified NDVI',opacity=0.85)
        

#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    
          
           
            
    
    if(dataset_selected=='LANDSAT'):
     
        dataset_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
                     .filter(period_of_interest1) \
                     .filterBounds(ROI) \
                     .first()
        
        ndvil = dataset_landsat.normalizedDifference(['B5', 'B4']).rename('NDVI')
              
        evil=dataset_landsat.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',  {'NIR': dataset_landsat.select("B5"), 'RED': dataset_landsat.select("B4"), 'BLUE': dataset_landsat.select("B2")})
        
         #Landsat
        ndvil = ndvil.select('NDVI').clip(ROI)
        
        dataset_landsat2 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
                     .filter(period_of_interest2) \
                     .filterBounds(ROI) \
                     .first()
        
        ndvil2 = dataset_landsat2.normalizedDifference(['B5', 'B4']).rename('NDVI')
              
        evil2=dataset_landsat2.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',  {'NIR': dataset_landsat.select("B5"), 'RED': dataset_landsat.select("B4"), 'BLUE': dataset_landsat.select("B2")})
        
         #Landsat
        ndvil2 =  ndvil2.select('NDVI').clip(ROI)
        
        change=ndvil2.subtract(ndvil)
                
        out_file2= directory + '/changel'+str(counter)+'.tif'
            
        geemap.ee_export_image(change, filename=out_file2)
       
        with changeout:
                clear_output()
            
                
                fp3 = (directory + '/changel'+str(counter)+'.tif')
                img3 = rasterio.open(fp3)
                display(show(img3))
        
        
   #Sentinel
    if(dataset_selected=='SENTINEL-2'):
       
        dataset_sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
                     .filter(period_of_interest1) \
                     .filterBounds(ROI) \
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                     .map(maskS2clouds)
    
    
        collection = dataset_sentinel.select(inBands).map(addNDVI).map(addEVI)

         #Sentinel
        band_median = collection.select(inBands).median()

         #Sentinel
        ndvimean = collection.select('NDVI').reduce(ee.Reducer.mean()).rename("NDVI")
        evimean = collection.select('EVI').reduce(ee.Reducer.mean()).rename("EVI")
        

        #sentinel
        ndvimean = ndvimean.clipToBoundsAndScale(ROI,1000,1000)
        evimean =  evimean.clipToBoundsAndScale(ROI,1000,1000)
        
        dataset_sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR') \
                     .filter(period_of_interest2) \
                     .filterBounds(ROI) \
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                     .map(maskS2clouds)
    
    
        collection2 = dataset_sentinel2.select(inBands).map(addNDVI).map(addEVI)

         #Sentinel
        band_median2 = collection.select(inBands).median()

         #Sentinel
        ndvimean2 = collection.select('NDVI').reduce(ee.Reducer.mean()).rename("NDVI")
        evimean2 = collection.select('EVI').reduce(ee.Reducer.mean()).rename("EVI")
        

        #sentinel
        ndvimean2 = ndvimean2.clipToBoundsAndScale(ROI,1000,1000)
       
        change=ndvimean.subtract(ndvimean2)
        ndviParams =  {'min': 1, 'max':5, 'palette' : ['654321','FFA500','FFFF00', '00FF00', '008000']}
        Map.addLayer(change,ndviParams)
                
        out_file2= directory + '/changes'+str(counter)+'.tif'
            
        geemap.ee_export_image(change, filename=out_file2)
       
        with changeout:
                clear_output()
                fp3 = (directory + '/changes'+str(counter)+'.tif')
                img3 = rasterio.open(fp3)
               
               
                display(show(img3))
        
        
       

   
    

In [ ]:
findchange.on_click(Changeindex)

In [ ]:
window=widgets.IntSlider(
    value=3,
    min=3,
    max=15,
    step=2,
    description='Window Size',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


In [ ]:

box=widgets.RadioButtons(
    options=['From Map', 'From Predefined Hyrcanian Forests'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='',
    disabled=False
)
out2=widgets.Output()
forest=widgets.VBox([step2or,Select_Forest])
def mytest5(x):         
        
             with out2:
                clear_output()
                if (box.value!='From Map'):
                    display(forest)
                    Map.centerObject(Dict['Abr'],6)
                    
                else:
                    Map.centerObject(Dict['Abr'],1)
                    Select_Forest.value = "None"
       
            



    
box.observe(mytest5, names='value')


In [ ]:

import ipywidgets as widgets
from IPython.core.display import display, HTML



douts.add_class("l")
display(HTML(
     "<style>.l {margin-top:30px;margin-l:10px;margin-r:0px;height:200px;width:350px;border-style: solid;}</style>"
))

verticalBox = mapWidget
heading=widgets.HTML(value='<h1 style="textalign:center;">RaoQ Index Calculator</h1>')
step1.layout.height = '35px'
dateBox.layout.height = '35px'
step2.layout.height = '35px'
getROI.layout.height = '35px'
step3.layout.height = '33px'
Select_Dataset.layout.height = '33px'
step4.layout.height = '35px'
DownloadRao_Btn.layout.height = '35px'
head=widgets.HBox([heading])
footerl=widgets.VBox([Display,Select_Output,douts])


footerr=widgets.VBox([step7,startd,endd,findchange,changeout])
footerr.add_class("left-spacing-class")
display(HTML(
     "<style>.left-spacing-class {margin-left:300px;}</style>"
))
footer=widgets.HBox([footerl,footerr])
footer.add_class("bottomspacing")
display(HTML(
     "<style>.bottomspacing {margin-top:50px;}</style>"
))

Display.layout.height = '40px'

DownloadRao_Btn.style.button_color='lightgreen'
getROI.style.button_color='lightgray'
hBox=widgets.HBox([step2,box]);
vBox1 = widgets.VBox([verticalBox])
vBox2 = widgets.VBox([step1, dateBox,step2,box,out2,
                      step3,Select_Dataset,step4,
                      index,outs,window,
                      step5,DownloadRao_Btn                  
                   ])
# step1, dateBox,step2, getROI,
AppLayout(header=head,
          left_sidebar=vBox1,
          right_sidebar=vBox2,
          footer=footer,
          height="80%", width="160%",
          pane_widths=[1.5, 0, 1.5],
          pane_heights=[.25,1.5,1])


In [ ]:
clear_output()

In [ ]:
!pip freeze > requirements.txt

In [ ]:
pip install voila


In [ ]:
voila final2.ipynb